In [ ]:
import os
import sys
sys.path.append(os.path.abspath('../'))

import pandas as pd
import requests
import lxml.html
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser

from Preprocess import *

In [ ]:
auth_key='YOURKEY'   # 여기에 전자공시시스템에서 발급받은 키를 넣어야하
start_date = '20090102' # ItemCharge에서 첫날짜
end_date  = '20190531' # ItemCharge에서 마지막 날짜

base_url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key

# Import & preprocess

In [ ]:
df_itemChargeWithIssue = pd.read_csv("../../data/fn_data_itemChargeWithIssue.csv", parse_dates=['date'])
PreprocessItemcode(df_itemChargeWithIssue)

# Trading stop item code

In [ ]:
stopitemcode = df_itemChargeWithIssue[df_itemChargeWithIssue['trade_stop_yn'] == 'Y']['itemcode'].unique()

In [ ]:
df_report = pd.DataFrame()
fail_code = []
for code in stopitemcode:
    url = base_url+"&crp_cd="+code+"&start_dt="+start_date+"&end_dt="+end_date+"&bsn_tp=I003"+"&page_set=100"+"&fin_rpt=N"
    
    resultXML = urlopen(url)  #this is for response of XML
    result = resultXML.read() #Using read method
    
    xmlsoup=BeautifulSoup(result,'html.parser')
    
    data = pd.DataFrame()

    te=xmlsoup.findAll("list")

    for t in te:
        if t.crp_nm.string == t.flr_nm.string:
            continue
        if not "거래정지" in t.rpt_nm.string:
            continue
        if "기타시장안내" in t.rpt_nm.string:
            continue
        if "기간변경" in t.rpt_nm.string:
            continue
        if "기간변경" in t.rpt_nm.string:
            continue
        if "주권매매거래정지해제" in t.rpt_nm.string:
            continue
        temp = pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
            t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
            columns = ["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data = pd.concat([data,temp])
    if len(data) == 0:
        fail_code.append(code)
    df_report = pd.concat([df_report, data])
df_report = df_report.reset_index(drop=True)

In [ ]:
def readTabel(report, doc_num, bPrintInfo = False):
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+df_report['rcp_no'][doc_num]
    req = requests.get(url)
    tree = lxml.html.fromstring(req.text)
    onclick = tree.xpath('//*[@id="north"]/div[2]/ul/li[1]/a')[0].attrib['onclick']
    pattern = re.compile("^openPdfDownload\('\d+',\s*'(\d+)'\)")
    dcm_no = pattern.search(onclick).group(1)
    url_parsing = 'http://dart.fss.or.kr/report/viewer.do?rcpNo='+df_report['rcp_no'][doc_num]+"&dcmNo="+dcm_no+'&eleId=0&offset=0&length=0&dtd=HTML'

    report=urlopen(url_parsing)
    r=report.read()

    xmlsoup_another = BeautifulSoup(r,'html.parser') 
    body=xmlsoup_another.find("body")
    table=body.find_all("table")
    p = parser.make2d(table[-1])[:3]
    
    if bPrintInfo:
        print(url)
        print(url_parsing)
        print(p)

    if len(p[0]) == 3:
        idx = 1
    else:
        idx = 2
    k = [[p[i][idx] for i in range(len(p))]]
    sheet = pd.DataFrame(k, columns=['itemname', 'trade_stop_reason', 'date'])

    sheet['itemcode'] = df_report['crp_cd'][doc_num]
    sheet['rcp_no'] = df_report['rcp_no'][doc_num]
    return sheet

In [ ]:
display(df_report[df_report['rpt_nm'] == '주권매매거래정지해제(투자자보호(감자 철회))'].head())
display(df_report[df_report['crp_nm'] == '이매진아시아'].head())

In [ ]:
df_report['rpt_nm'].unique()

In [ ]:
stop_reason_sheet = pd.DataFrame()
for doc_num in range(len(df_report)):
    sheet = readTabel(df_report, doc_num)
    stop_reason_sheet = pd.concat([stop_reason_sheet, sheet])
stop_reason_sheet = stop_reason_sheet.reset_index(drop=True)
stop_reason_sheet = stop_reason_sheet[stop_reason_sheet['date'].apply(len) >= 10]
stop_reason_sheet.head()

In [ ]:
df_report.head()

In [ ]:
readTabel(df_report, 1, True)

In [ ]:
stop_reason_sheet['date'] = stop_reason_sheet['date'].map(lambda x: x.rstrip('부터').replace("년 ", "-",1).replace("월 ", "-",1).replace("일", "",1)[:10])
stop_reason_sheet['date'] = pd.to_datetime(stop_reason_sheet['date'])

In [ ]:
stop_reason_sheet.tail()

# 중복 보도 제거

In [ ]:
print(len(stop_reason_sheet))
stop_reason_sheet['duplicate_cnt'] = 0

idxs = []
for df in stop_reason_sheet.groupby('itemcode'):
    df = pd.DataFrame(df[1])
    b = False
    
    for df2 in df.groupby('date'):
        if len(df2[1]) > 1:
            idxs += list(df2[1].index)[1:]
            b = True
        stop_reason_sheet.loc[df2[1].index, 'duplicate_cnt'] = len(df2[1])
stop_reason_sheet = stop_reason_sheet.drop(idxs)
print(len(stop_reason_sheet))

In [ ]:
stop_reason_sheet[stop_reason_sheet['itemcode'] == '001260']

In [ ]:
readTabel(df_report, 3, True)

# File save

In [ ]:
stop_reason_sheet.to_csv("./TradeStopReason.csv", index=False)

In [ ]:
stop_reason_sheet[stop_reason_sheet['trade_stop_reason'] == '풍문 등 조회결과 공시\r\n반기감사보고서 제출']

In [ ]:
stop_reason_sheet['trade_stop_reason'].unique()

In [ ]:
merge3 = mergeTradeStopReason(df_itemChargeWithIssue, stop_reason_sheet)

In [ ]:
merge3.head()

In [ ]:
merge3.to_csv("../../data/fn_data_itemChargeWithIssueStopReason.csv", index=False)

In [ ]:
print(merge3['trade_stop_reason'].unique())
print(len(merge3['trade_stop_reason'].unique()))